In [1]:
from miditoolkit.midi.parser import MidiFile

from deepnote import MusicRepr, Constants, Metric, Note, utils

## constants

In [2]:
const = Constants()
const

Constants(unit=12, tick_resol=120, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)

In [3]:
vars(const).keys()

dict_keys(['unit', 'n_bar_steps', 'tick_resol', 'step', 'bar_resol', 'position_bins', 'min_tempo', 'max_tempo', 'num_tempo_bins', 'tempo_bins', 'pitch_classes', 'chord_qualities', 'chords', 'instruments', 'pitches', 'duration_bins', 'num_velocity_bins', 'velocity_bins', 'all_tokens'])

## midi representation

In [4]:
seq = MusicRepr.from_file(
    '../test-1.mid', 
    unit=4,
    min_tempo=30,
    max_tempo=300,
    num_tempo_bins=30, 
    num_velocity_bins=30
)
seq

MusicRepr(
 num_events=799,
 const=Constants(unit=4, tick_resol=100, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)
)

In [5]:
seq.const

Constants(unit=4, tick_resol=100, min_tempo=30, max_tempo=300, num_tempo_bins=30, num_velocity_bins=30)

In [6]:
seq[:10]

[Bar(),
 Beat(position=1, tempo=169),
 Beat(position=2),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=83),
 Note(inst_family=guitar, pitch=54, duration=3, velocity=53),
 Beat(position=4),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=57),
 Note(inst_family=bass, pitch=35, duration=1, velocity=105),
 Beat(position=6),
 Note(inst_family=guitar, pitch=59, duration=3, velocity=61)]

### splitting bars

In [7]:
seq.get_bar_count()

18

In [8]:
bars = seq.get_bars()
bars[0][:]

[Bar(),
 Beat(position=1, tempo=169),
 Beat(position=2),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=83),
 Note(inst_family=guitar, pitch=54, duration=3, velocity=53),
 Beat(position=4),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=57),
 Note(inst_family=bass, pitch=35, duration=1, velocity=105),
 Beat(position=6),
 Note(inst_family=guitar, pitch=59, duration=3, velocity=61),
 Note(inst_family=guitar, pitch=62, duration=3, velocity=61),
 Note(inst_family=guitar, pitch=66, duration=3, velocity=53),
 Note(inst_family=guitar, pitch=54, duration=7, velocity=57),
 Note(inst_family=bass, pitch=30, duration=3, velocity=113),
 Beat(position=9),
 Note(inst_family=guitar, pitch=62, duration=1, velocity=66),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=74),
 Note(inst_family=guitar, pitch=59, duration=3, velocity=66),
 Beat(position=11),
 Note(inst_family=guitar, pitch=66, duration=3, velocity=57),
 Note(inst_family=guitar, pitch=62, duration=5, velocity=44),


## concatenating bars

In [9]:
seq2 = MusicRepr.concatenate(bars[:2])
seq2.get_bar_count()

2

## Slicing

### bar index

In [10]:
seq2 = seq.slice_by_bar(1,2)
print(seq2.events == bars[1].events, seq2 == bars[1])
seq2[:10]

True True


[Bar(),
 Beat(position=2),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=57),
 Note(inst_family=bass, pitch=35, duration=1, velocity=92),
 Beat(position=4),
 Note(inst_family=guitar, pitch=59, duration=3, velocity=61),
 Note(inst_family=guitar, pitch=62, duration=3, velocity=61),
 Note(inst_family=guitar, pitch=66, duration=3, velocity=53),
 Note(inst_family=guitar, pitch=54, duration=7, velocity=57),
 Note(inst_family=bass, pitch=35, duration=1, velocity=118)]

### beat index

In [11]:
seq2 = seq.slice_by_beat(12, 16)
print(seq2.events == bars[3].events, seq2 == bars[3])
seq2[:10]

True True


[Bar(),
 Beat(position=4),
 Note(inst_family=guitar, pitch=61, duration=1, velocity=83),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=92),
 Note(inst_family=guitar, pitch=58, duration=3, velocity=83),
 Note(inst_family=bass, pitch=30, duration=2, velocity=105),
 Note(inst_family=piano, pitch=70, duration=2, velocity=70),
 Note(inst_family=synth-effects, pitch=70, duration=2, velocity=31),
 Beat(position=5),
 Note(inst_family=guitar, pitch=61, duration=1, velocity=53)]

### event index

In [12]:
seq2 = seq.slice_by_index(0,10)
seq2[:]

[Bar(),
 Beat(position=1, tempo=169),
 Beat(position=2),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=83),
 Note(inst_family=guitar, pitch=54, duration=3, velocity=53),
 Beat(position=4),
 Note(inst_family=guitar, pitch=66, duration=1, velocity=57),
 Note(inst_family=bass, pitch=35, duration=1, velocity=105),
 Beat(position=6),
 Note(inst_family=guitar, pitch=59, duration=3, velocity=61)]

## separating different instrument tracks

In [13]:
tracks = seq.separate_tracks()
list(tracks.keys()), seq.get_instruments()

(['piano', 'bass', 'guitar', 'synth-effects'],
 ['piano', 'bass', 'guitar', 'synth-effects'])

In [14]:
for inst in tracks:
    print(tracks[inst].get_bar_count())

18
18
18
18


In [15]:
tracks['bass'][:10]

[Bar(),
 Beat(position=1, tempo=169),
 Beat(position=4),
 Note(inst_family=bass, pitch=35, duration=1, velocity=105),
 Beat(position=6),
 Note(inst_family=bass, pitch=30, duration=3, velocity=113),
 Beat(position=11),
 Note(inst_family=bass, pitch=30, duration=1, velocity=100),
 Beat(position=13),
 Note(inst_family=bass, pitch=35, duration=4, velocity=118)]

## exporting

### midi

In [16]:
midi = seq.to_midi('test.mid')
midi

ticks per beat: 100
max tick: 7025
tempo changes: 1
time sig: 1
key sig: 0
markers: 0
lyrics: False
instruments: 4

### remi

In [17]:
remi = seq.to_remi(ret='token')
remi[:10]

['Bar',
 'BeatPosition_1',
 'BeatTempo_169',
 'BeatPosition_2',
 'NoteInstFamily_guitar',
 'NotePitch_66',
 'NoteDuration_1',
 'NoteVelocity_83',
 'NoteInstFamily_guitar',
 'NotePitch_54']

In [18]:
seq.to_remi(ret='event')[:10]

[Event(type=Bar),
 Event(type=BeatPosition, value=1),
 Event(type=BeatTempo, value=169),
 Event(type=BeatPosition, value=2),
 Event(type=NoteInstFamily, value=guitar),
 Event(type=NotePitch, value=66),
 Event(type=NoteDuration, value=1),
 Event(type=NoteVelocity, value=83),
 Event(type=NoteInstFamily, value=guitar),
 Event(type=NotePitch, value=54)]

In [19]:
seq.to_remi(ret='index')[:10]

[0, 2, 33, 3, 183, 263, 325, 360, 183, 251]

### compound word

In [20]:
cp = seq.to_cp()
cp.shape, len(seq)

((799, 8), 799)

In [21]:
bars[-1].to_cp()

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  3, 66,  0, 13],
       [ 1,  0,  0,  0,  4, 35,  0, 26],
       [ 0,  3,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  3, 62,  2, 17],
       [ 1,  0,  0,  0,  3, 66,  2, 14],
       [ 1,  0,  0,  0,  4, 33,  0, 26],
       [ 1,  0,  0,  0,  0, 78,  0,  9],
       [ 1,  0,  0,  0,  0, 74,  1, 24],
       [ 1,  0,  0,  0, 12, 78,  0,  3],
       [ 1,  0,  0,  0, 12, 74,  1, 10],
       [ 0,  4,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  4, 37,  0, 24],
       [ 1,  0,  0,  0,  0, 73,  1, 24],
       [ 1,  0,  0,  0,  0, 76,  1,  8],
       [ 1,  0,  0,  0, 12, 73,  1, 10],
       [ 1,  0,  0,  0, 12, 76,  1,  3],
       [ 0,  6,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  3, 66,  0, 18],
       [ 1,  0,  0,  0,  3, 62,  2, 21],
       [ 1,  0,  0,  0,  4, 38,  0, 21],
       [ 0,  8,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  4, 39,  0, 21]])

## Importing

### from cp

In [22]:
seq_t = MusicRepr.from_cp(cp, const=seq.const)
seq_t.events == seq.events, seq == seq_t

(True, True)

### from remi

In [24]:
seq_t = MusicRepr.from_string(' '.join(remi), const=seq.const)
seq_t.events == seq.events, seq == seq_t

(True, True)

## merging tracks

In [25]:
seq_t = MusicRepr.merge_tracks(tracks)
seq == seq_t

True